In [1]:
import numpy as np
import pandas as pd 
import os
from pandas.tseries.holiday import USFederalHolidayCalendar
from glob import glob
from collections import defaultdict

import utils
# from utils import load_data, get_train_val_split, get_stratified_splitter
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedGroupKFold, train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, make_scorer

from lightgbm import LGBMRegressor
from scipy.stats import kstest, kruskal, mannwhitneyu
from scipy.special import softmax 
from itertools import combinations
from collections import defaultdict
from tqdm import tqdm
from sklearn.ensemble import StackingRegressor
from sklearn.base import BaseEstimator, RegressorMixin
import re
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_log_error, mean_squared_error

import optuna
import pickle

In [2]:
import importlib
importlib.reload(utils)

<module 'utils' from '/Users/sharaddargan/Documents/Course Content/Spring 2023/Probability and Statistics 2/Project/EnergyPrediction-ASHRAE/code/utils.py'>

In [3]:
data_dict = utils.load_data('ashrae-energy-prediction')

Memory usage of dataframe is 0.07 MB
Memory usage after optimization is: 0.02 MB
Decreased by 73.88%
Memory usage of dataframe is 9.60 MB
Memory usage after optimization is: 3.07 MB
Decreased by 68.05%


/Users/sharaddargan/Documents/Course Content/Spring 2023/Probability and Statistics 2/Project/EnergyPrediction-ASHRAE/code/utils.py:143: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead
  weather_train['timestamp'] = pd.to_datetime(weather_train['timestamp'], infer_datetime_format = True, utc = True).astype('datetime64[ns]')


Memory usage of dataframe is 19.04 MB
Memory usage after optimization is: 5.13 MB
Decreased by 73.04%


/Users/sharaddargan/Documents/Course Content/Spring 2023/Probability and Statistics 2/Project/EnergyPrediction-ASHRAE/code/utils.py:151: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead
  train['timestamp'] = pd.to_datetime(train['timestamp'], infer_datetime_format = True, utc = True).astype('datetime64[ns]')


Memory usage of dataframe is 616.95 MB
Memory usage after optimization is: 289.19 MB
Decreased by 53.12%
Memory usage of dataframe is 1272.51 MB
Memory usage after optimization is: 358.53 MB
Decreased by 71.82%


/Users/sharaddargan/Documents/Course Content/Spring 2023/Probability and Statistics 2/Project/EnergyPrediction-ASHRAE/code/utils.py:166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[(train['site_id'] == 0) & (train['meter'] == 0)]['meter_reading'] = 0.2931 * train[(train['site_id'] == 0) & (train['meter'] == 0)]['meter_reading']


In [4]:
# Add weather features 
weather_features = ['cloud_coverage', 'dew_temperature', 'air_temperature', 
                    'sea_level_pressure', 'wind_direction', 'wind_speed', 'precip_depth_1_hr',]

hourly_by_site = data_dict["X_train"].groupby(['hour', 'month', 'site_id'])[weather_features].mean().reset_index()

data_dict["X_train"] = data_dict["X_train"].merge(
    hourly_by_site, 
    on=['hour', 'month', 'site_id'], 
    how='left', 
    suffixes=(None, '_hourly_by_site')
)

del hourly_by_site

for feature in weather_features:
    # Fill in NA values from weather with hourly by site columns 
    data_dict["X_train"][feature].fillna(
        data_dict["X_train"][feature + "_hourly_by_site"],
        inplace=True
    )
    
    # Fill in the rest with the median 
    data_dict["X_train"][feature].fillna(
        data_dict["X_train"][feature].median(),
        inplace=True
    )
    
    data_dict["X_train"][feature + "_diff_hourly_from_mean"] = data_dict["X_train"][feature] - \
        data_dict["X_train"][feature + "_hourly_by_site"]
    
data_dict["X_train"] = data_dict["X_train"].drop(columns = [feat + "_hourly_by_site" for feat in weather_features])

In [5]:
# Fill in NA with median values for floor count and year_built
for feature in ['year_built', 'floor_count']:
    data_dict["X_train"][feature].fillna(
        data_dict["X_train"][feature].median(), 
        inplace=True
    )

In [6]:
data_dict.keys()

dict_keys(['weather_test', 'X_train', 'X_test', 'y_train'])

## Examine Differences (Non-Parametric)
Using Bonferonni's Correction

### Milestone 2. 
- Show difference in sites across meter readings 
- get average meter reading per day per site 
- conduct a (non-parametric ANOVA) KS OR pairs (mann-whitney) to show that they are diff 
- train a model per site id (with rudimentary hyperparameter tuning) 
- John sites 0-7, Sharad sites 8-15 

### Milestone 3. Determine, per site, which primary uses are similar (if they have only a few buildings), which are diff
- for a given primary use, if diff, identify "clusters" of buildings that are similar 

In [7]:
features = ['year_built', 'floor_count', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
       'sea_level_pressure', 'wind_direction', 'wind_speed',
       'air_temperature_mean_lag7', 'air_temperature_max_lag7',
       'air_temperature_min_lag7', 'air_temperature_std_lag7',
       'cloud_coverage_mean_lag7', 'cloud_coverage_max_lag7',
       'cloud_coverage_min_lag7', 'cloud_coverage_std_lag7',
       'dew_temperature_mean_lag7', 'dew_temperature_max_lag7',
       'dew_temperature_min_lag7', 'dew_temperature_std_lag7',
       'precip_depth_1_hr_mean_lag7', 'precip_depth_1_hr_max_lag7',
       'precip_depth_1_hr_min_lag7', 'precip_depth_1_hr_std_lag7',
       'sea_level_pressure_mean_lag7', 'sea_level_pressure_max_lag7',
       'sea_level_pressure_min_lag7', 'sea_level_pressure_std_lag7',
       'wind_direction_mean_lag7', 'wind_direction_max_lag7',
       'wind_direction_min_lag7', 'wind_direction_std_lag7',
       'wind_speed_mean_lag7', 'wind_speed_max_lag7', 'wind_speed_min_lag7',
       'wind_speed_std_lag7', 'log_square_feet', 'weekday', 'hour', 'day',
       'weekend', 'month', 'primary_use_enc']

In [8]:
#building_cluster_mapping = pd.read_csv("building_cluster_mapping.csv")

In [8]:
X, y = data_dict['X_train'].loc[:, features+['meter', 'site_id', 'building_id']], data_dict['y_train']

In [10]:
#X = X.reset_index().merge(building_cluster_mapping, on=["building_id", "meter"], how="left").set_index('index')

In [9]:
splitter_gen = utils.get_stratified_splitter(X, y)
train_index, test_index = next(splitter_gen)
X_train, y_train, X_test, y_test = X.loc[train_index, :], y[train_index], X.loc[test_index, :], y[test_index]

/opt/homebrew/Caskroom/miniforge/base/envs/conda-env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:909: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(


In [10]:
model_dir = "./models/v4/"
models = defaultdict(list)
for m in os.listdir(model_dir):
    if '.pkl' in m:
        with open(model_dir + m, 'rb') as f:
            models[int(m.split("_")[1])].append(pickle.load(f))

In [15]:
# Saved as "site_meter"
class WeightedEnsembleRegressor(BaseEstimator, RegressorMixin):
    # Store weights per site per meter
    def __init__(self, models: dict, chunk_size: int = 1000000):
        self.models = models
        self.weights = {}
        self.chunk_size = chunk_size
        
        for i in range(4):
            self.weights[i] = np.ones(len(self.models[i])) / len(self.models[i])
        
    def fit(self, X, y):
        new_weights = self.weights.copy()
        
        for i in range(4): 
            new_weights[i] = np.zeros(len(self.weights[i]))
        
        for chunk in range(0, len(X), self.chunk_size):
            for meter in X['meter'].unique():

                for i in range(len(self.models[meter])):
                    try:
                        prediction = self.models[meter][i].predict(
                            X.loc[chunk: chunk + self.chunk_size][X['meter'] == meter] \
                                .drop(['meter', 'site_id', 'building_id'], axis=1)
                        )
                        new_weights[meter][i] += mean_squared_error(
                            prediction, 
                            y.iloc[chunk: chunk + self.chunk_size][X['meter'] == meter]
                        )
                    except:
                        continue
            
        for i in range(4): 
            self.weights[i] = softmax(-new_weights[i])
            
    
    def predict(self, X):
        preds = np.zeros(len(X))
        for chunk in range(0, len(X), self.chunk_size):
            for meter in X['meter'].unique():
                for i in range(len(self.models[meter])):
                    prediction = self.models[meter][i].predict(
                    X.iloc[chunk: chunk + self.chunk_size][X['meter'] == meter] \
                        .drop(['meter', 'site_id', 'building_id'], axis=1)
                    )
                    preds[chunk: chunk + self.chunk_size][X['meter'] == meter] += self.weights[meter][i] * prediction
  
                    
        return preds

In [21]:
# Saved as "site_meter"
class WeightedEnsembleRegressor(BaseEstimator, RegressorMixin):
    # Store weights per site per meter
    def __init__(self, models: dict, chunk_size: int = 1000000):
        self.models = models
        self.weights = {}
        self.chunk_size = chunk_size
        
        for i in range(4):
            self.weights[i] = np.ones(len(self.models[i])) / len(self.models[i])
        
    def fit(self, X, y):
        new_weights = self.weights.copy()
        
        for i in range(4): 
            new_weights[i] = np.zeros(len(self.weights[i]))
        
        for chunk in range(0, len(X), self.chunk_size):
            for meter in X['meter'].unique():

                for i in range(len(self.models[meter])):
                    try:
                        prediction = self.models[meter][i].predict(
                            X.loc[chunk: chunk + self.chunk_size,:].loc[X['meter'] == meter, :] \
                                .drop(['meter', 'site_id', 'building_id'], axis=1)
                        )
                        new_weights[meter][i] += mean_squared_error(
                            prediction, 
                            y.iloc[chunk: chunk + self.chunk_size][X['meter'] == meter]
                        )
                    except:
                        continue
            
        for i in range(4): 
            self.weights[i] = softmax(-new_weights[i])
            
    
    def predict(self, X):
        preds = np.zeros(len(X))
        for meter in X['meter'].unique():
            for i in range(len(self.models[meter])):
                prediction = self.models[meter][i].predict(
                X.loc[X['meter'] == meter, :].drop(['meter', 'site_id', 'building_id'], axis=1)
                )
                preds[X['meter'] == meter] += self.weights[meter][i] * prediction
                    
        return preds

In [22]:
w = WeightedEnsembleRegressor(models)
w.fit(X_train.reset_index(drop=True), y_train.reset_index(drop=True))

In [24]:
def predict(self, X):
    preds = np.zeros(len(X))
    for meter in X['meter'].unique():
        for i in range(len(self.models[meter])):
            prediction = self.models[meter][i].predict(
            X.loc[X['meter'] == meter, :].drop(['meter', 'site_id', 'building_id'], axis=1)
            )
            preds[X['meter'] == meter] += self.weights[meter][i] * prediction
    return preds

In [25]:
mean_squared_error(y_train.reset_index(drop=True), predict(w, X_train.reset_index(drop=True)), squared=False)

1.7918102143022188

In [26]:
mean_squared_error(y_test.reset_index(drop=True), predict(w, X_test.reset_index(drop=True)), squared=False)

1.8038478151090174

In [ ]:
results_df = predict_using_individual_estimators(X_test, y_test, estimators_dict)

In [ ]:
results_df = predict_using_individual_estimators(X_train, y_train, estimators_dict)